In [1]:
import math
import json
import csv
import pandas as pd
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import os
import glob
import datetime
import copy
from random import shuffle
import random

In [2]:
#one bin means no seperation
#two bins means seperation based on up/down
#n bins partitions [-3,3] into n-2 different equal sections with a bin for [-inf,-3] and [3,inf]
#returns an ordered pair representing the bin
def get2DBin(percents, numBins):
    #hardCode 1 bin and 2 bins
    if numBins == 1:
        return (0,0)
    elif numBins == 2:
        return (int(percents[0] >0), int(percents[1]> 0))
    else:
        daily = 0
        weekly = 0
        binWidth = 6/(numBins-2)
        binHeight = 12/(numBins-2)
        for i in range(numBins-1):
            if percents[0] > -3+binWidth*i:
                daily = i+1
            if percents[1] > -6+binHeight*i:
                weekly = i+1
    return daily, weekly

In [3]:
#result is the current stocks change in percent - the ops stock change in percent
def upDateElo(currentElo, opsElo, result, flexibility):
    variationFromResult = result-(currentElo-opsElo)
    return currentElo + flexibility*variationFromResult

In [3]:
#returns a dictionary each stock name is the key and value is 2d array row is 1 day, col is 5 day
def createPlayers(stocks, bins):
    stocks_lower = [stock.lower() for stock in stocks]
    consolidated = {}
    if len(stocks) == 0:
        for filename in os.listdir("./SP500Stocks"):
            if "csv" in filename:
                consolidated[filename[:-4]] = [[0]*bins]*bins
    else:
        for filename in os.listdir("./SP500Stocks"):
            if "csv" in filename and filename[:-4].lower() in stocks_lower:
                consolidated[filename[:-4]] = [[0]*bins]*bins
    return consolidated
#createPlayers([],2)

In [6]:
def returns(directory, stock):
    df = pd.read_csv(os.path.join(directory, stock), usecols=["Date", "Adj Close"], parse_dates=True)
    df["daily_returns"] = df["Adj Close"].diff().astype("float64")
    df["daily_returns"] = 100 * df["daily_returns"] / df["Adj Close"].shift(1)
    df["5_day_returns"] = df["daily_returns"].rolling(5).apply(lambda x: 100 * (np.prod(1 + x / 100) - 1), raw=False).astype("float64")
    df["Date"] = df["Date"].astype(str)
    df.dropna(inplace=True)
    df.drop(columns="Adj Close", inplace=True)
    df.columns = [stock[:-4] + "_" + col if col != "Date" else col for col in df.columns]
    return df

In [10]:
#merge all stocks together
directory = "SmallStocks"
files = os.listdir(directory)
master = returns(directory,files[0])

for filename in files[1:]:
    if "csv" in filename:
        master = master.merge(returns(directory,filename), on="Date")
master.head()
master.shape

(2408, 23)

In [9]:
##Now for the main script
#lets see our profits if we bet same amount every single time on the favorite
def binary_unweighted_bet_strat(master,stocks, bins, flexibility):
    players = createPlayers(stocks, bins)
    trading_stocks = players.keys()
    #daily profits are the cumulative profits we have each day from all of our trades
    #day profits are profits that we have within each day weighted by how much money we put on
    #since this strat is unweighted for now we will set all the weights to be 1
    daily_profits = [0]
    day_profits = []
    for i in range(master.shape[0]-1):
        for stock_1 in trading_stocks:
            print(stock_1)
            print(master.iloc[i][stock_1+"_daily_returns"])
            print(master.iloc[i][stock_1+"_5_day_returns"])
            numCollisions = 0
            for stock_2 in trading_stocks:
                if stock_1 != stock_2 and get2DBin((master.iloc[i][stock_1+"_daily_returns"],
                             master.iloc[i][stock_1+"_5_day_returns"]), bins) == get2DBin((master.iloc[i][stock_2+"_daily_returns"],
                             master.iloc[i][stock_2+"_5_day_returns"]), bins):
                    print(stock_1)
                    print(stock_2)
                
                #continue
                #print(stock_1)
                #print(master.iloc[i][stock_1+"_daily_returns"])
                #print(master.iloc[i][stock_1+"_5_day_returns"])
        return
        print (i)
binary_unweighted_bet_strat(master, [],5,.01)      

CSCO


KeyError: 'CSCO_daily_returns'